In [1]:
import argparse
import os
import time
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
from graphsaint.sampler import SAINTNodeSampler, SAINTEdgeSampler, SAINTRandomWalkSampler
from graphsaint.config import CONFIG
from graphsaint.modules import GCNNet
from graphsaint.utils import Logger, evaluate, save_log_dir, load_data, calc_f1
import warnings

graphsaint > tran_sampling.py > main()

In [3]:
a = {
        'aggr': 'concat', 'arch': '1-0-1-0', 'dataset': 'ppi', 'dropout': 0, 'edge_budget': 4000, 'length': 2,
        'log_dir': 'none', 'lr': 0.01, 'n_epochs': 50, 'n_hidden': 512, 'no_batch_norm': False, 'node_budget': 6000,
        'num_subg': 50, 'num_roots': 3000, 'sampler': 'node', 'use_val': True, 'val_every': 1, 'num_workers_sampler': 0,
        'num_subg_sampler': 10000, 'batch_size_sampler': 200, 'num_workers': 8, 'full': True, 'online': False, 'gpu': 0,
}
multilabel =  True

from collections import namedtuple
A = namedtuple('a', a)
args = A(**a)

In [4]:
# load and preprocess dataset
data = load_data(args, multilabel)
g = data.g
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
labels = g.ndata['label']

train_nid = data.train_nid

in_feats = g.ndata['feat'].shape[1]
n_classes = data.num_classes
n_nodes = g.num_nodes()
n_edges = g.num_edges()

n_train_samples = train_mask.int().sum().item()
n_val_samples = val_mask.int().sum().item()
n_test_samples = test_mask.int().sum().item()

In [5]:
data

Dataset(num_classes=121, train_nid=array([   0,    1,    2, ..., 9713, 9714, 9715]), g=Graph(num_nodes=14755, num_edges=450540,
      ndata_schemes={'feat': Scheme(shape=(50,), dtype=torch.float32), 'label': Scheme(shape=(121,), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={}))

In [6]:
kwargs = {
        'dn': args.dataset, 'g': g, 'train_nid': train_nid, 'num_workers_sampler': args.num_workers_sampler,
        'num_subg_sampler': args.num_subg_sampler, 'batch_size_sampler': args.batch_size_sampler,
        'online': args.online, 'num_subg': args.num_subg, 'full': args.full
    }

In [7]:
if args.sampler == "node":
        saint_sampler = SAINTNodeSampler(args.node_budget, **kwargs)
elif args.sampler == "edge":
    saint_sampler = SAINTEdgeSampler(args.edge_budget, **kwargs)
elif args.sampler == "rw":
    saint_sampler = SAINTRandomWalkSampler(args.num_roots, args.length, **kwargs)
else:
    raise NotImplementedError
loader = DataLoader(saint_sampler, collate_fn=saint_sampler.__collate_fn__, batch_size=1,
                    shuffle=True, num_workers=args.num_workers, drop_last=False)

[3.1428573 2.9642859 3.        ... 3.        3.2807019 3.1612902]
[0.00028198 0.00228718 0.00027085 ... 0.00036113 0.00128654 0.00033201]
The number of subgraphs is:  200


In [8]:
# set device for dataset tensors
cpu_flag = False

if args.gpu < 0:
    cuda = False
else:
    cuda = True
    torch.cuda.set_device(args.gpu)
    val_mask = val_mask.cuda()
    test_mask = test_mask.cuda()
    if not cpu_flag:
        g = g.to('cuda:{}'.format(args.gpu))

print('labels shape:', g.ndata['label'].shape)
print("features shape:", g.ndata['feat'].shape)

model = GCNNet(
    in_dim=in_feats,
    hid_dim=args.n_hidden,
    out_dim=n_classes,
    arch=args.arch,
    dropout=args.dropout,
    batch_norm=not args.no_batch_norm,
    aggr=args.aggr
)

if cuda:
    model.cuda()

# logger and so on
log_dir = save_log_dir(args)
logger = Logger(os.path.join(log_dir, 'loggings'))
logger.write(args)

# use optimizer
optimizer = torch.optim.Adam(model.parameters(),
                                lr=args.lr)

# set train_nids to cuda tensor
if cuda:
    train_nid = torch.from_numpy(train_nid).cuda()
    print("GPU memory allocated before training(MB)",
            torch.cuda.memory_allocated(device=train_nid.device) / 1024 / 1024)
start_time = time.time()
best_f1 = -1

labels shape: torch.Size([14755, 121])
features shape: torch.Size([14755, 50])
a(aggr='concat', arch='1-0-1-0', dataset='ppi', dropout=0, edge_budget=4000, length=2, log_dir='none', lr=0.01, n_epochs=50, n_hidden=512, no_batch_norm=False, node_budget=6000, num_subg=50, num_roots=3000, sampler='node', use_val=True, val_every=1, num_workers_sampler=0, num_subg_sampler=10000, batch_size_sampler=200, num_workers=8, full=True, online=False, gpu=0)
GPU memory allocated before training(MB) 26.75830078125


In [21]:
task = 1 ######## alternative to Argparser

for epoch in range(args.n_epochs):
    for j, subg in enumerate(loader):
        if cuda:
            subg = subg.to(torch.cuda.current_device())
        model.train()
        # forward
        pred = model(subg)
        batch_labels = subg.ndata['label']

        if multilabel:
            loss = F.binary_cross_entropy_with_logits(pred, batch_labels, reduction='sum',
                                                        weight=subg.ndata['l_n'].unsqueeze(1))
        else:
            loss = F.cross_entropy(pred, batch_labels, reduction='none')
            loss = (subg.ndata['l_n'] * loss).sum()

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), 5)
        optimizer.step()

        if j == len(loader) - 1:
            model.eval()
            with torch.no_grad():
                train_f1_mic, train_f1_mac = calc_f1(batch_labels.cpu().numpy(),
                                                        pred.cpu().numpy(), multilabel)
                print(f"epoch:{epoch + 1}/{args.n_epochs}, Iteration {j + 1}/"
                        f"{len(loader)}:training loss", loss.item())
                print("Train F1-mic {:.4f}, Train F1-mac {:.4f}".format(train_f1_mic, train_f1_mac))
    # evaluate
    model.eval()
    if epoch % args.val_every == 0:
        # if cpu_flag and cuda:  # Only when we have shifted model to gpu and we need to shift it back on cpu
        #     model = model.to('cpu')
        #     val_mask = val_mask.to('cpu')
        # model = model.to(torch.cuda.current_device())
        labels = labels.to(torch.cuda.current_device()) # ignore
        
        val_f1_mic, val_f1_mac = evaluate(
            model, g, labels, val_mask, multilabel)
        print(
            "Val F1-mic {:.4f}, Val F1-mac {:.4f}".format(val_f1_mic, val_f1_mac))
        if val_f1_mic > best_f1:
            best_f1 = val_f1_mic
            print('new best val f1:', best_f1)
            torch.save(model.state_dict(), os.path.join(
                log_dir, 'best_model_{}.pkl'.format(task)))
        if cpu_flag and cuda:
            model.cuda()

end_time = time.time()
print(f'training using time {end_time - start_time}')

c:\Users\Laurelwoods\AppData\Local\Programs\Python\Python311\Lib\site-packages\dgl\backend\pytorch\tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (
C:\Users\Laurelwoods\AppData\Local\Temp\ipykernel_1068\3363985756.py:21: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), 5)


epoch:1/50, Iteration 200/200:training loss 4.100785255432129
Train F1-mic 0.9926, Train F1-mac 0.9911


c:\Users\Laurelwoods\AppData\Local\Programs\Python\Python311\Lib\site-packages\dgl\backend\pytorch\tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


Val F1-mic 0.9690, Val F1-mac 0.9644
new best val f1: 0.9690338883751792


KeyboardInterrupt: 